In [ ]:
from datasets import load_dataset

dataset = load_dataset("HelpingAI/Intermediate-Thinking-130k", split='train')

In [ ]:
import re
def format_dataset(example):
    instruction = example['instruction']
    output = example['output']

    # Remove ser blocks
    output = re.sub(r'<ser>.*?</ser>', '', output, flags=re.DOTALL)
    
    # Use a regular expression to match the entire block starting with <think> and ending with the explanation after </think>
    think_blocks = re.findall(r'(<think>.*?</think>.*?)(?=\n<think>|$)', output, re.DOTALL)
    
    # Create a list of messages
    converted_sample = [
            {"role": "user", "content": instruction},
        ]
    
    for block in think_blocks:
        converted_sample.append({"role": "assistant", "content": block})

    return {'messages': converted_sample}

In [ ]:
dataset = dataset.map(format_dataset, num_proc=8)
dataset = dataset.remove_columns(['instruction','input','output','conversation'])

In [ ]:
pattern = r"<ser>.*?</ser>"

for messages in dataset['messages']:
    for sample in messages:
        # Check if the pattern exists in the sample text
        if re.search(pattern, sample['content'], re.DOTALL):  # re.DOTALL allows '.' to match newlines
            print("A <ser> block exists in the text.")